In [1]:
from qiskit import QuantumCircuit, transpile, execute, IBMQ
from states import Key, Ciphertext
from global_parameters import GlobalParameters
from encryption_circuit import encrypt
from verification_circuit import verify_deletion_counts
from decryption_circuit import create_decryption_circuit, decrypt_results
from deletion_circuit import delete
from attack_circuit import breidbart_measurement
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeMontreal
from qiskit.circuit import qpy_serialization
from utils import export_counts, random_bit_string
import os
import shutil

In [2]:
%%capture cap --no-stderr

# Qiskit parameters
provider = IBMQ.load_account()

# Ideal simulator
# backend = AerSimulator()
# print("Ideal simulator AerSimulator")

# Simulator
noise_model = FakeMontreal()
backend = AerSimulator.from_backend(noise_model)
print("Simulator with noise model from ibmq_montreal")

# Real backend
# backend = provider.get_backend("ibm_nairobi")
# print("Real backend ibm_nairobi")

optimization_level = 0
shots = 1000

# Create data directory
if not os.path.exists("data"):
    os.mkdir("data")

notebook_output_filepath = "data/notebook_output.txt"
# Replace the notebook output text file with a blank file
with open(notebook_output_filepath, "w") as f:
    pass

def write_cell_to_file(string_to_write: str) -> None:
    """"Writes the cell output to the globally-defined notebook_output_filepath file."""
    with open(notebook_output_filepath, "a") as f:
        f.write(string_to_write + "\n")

In [3]:
write_cell_to_file(cap.stdout)

In [4]:
def run_and_measure(circuit: QuantumCircuit, draw: bool = False) -> dict[str, int]:
    """Transpiles and runs the QuantumCircuit, and returns the resulting counts of the execution."""
    if draw:
        print(circuit.draw())
    transpiled_circuit = transpile(
        circuit, backend=backend, optimization_level=optimization_level)
    if draw:
        print(transpiled_circuit.draw())
    result = backend.run(transpiled_circuit, shots=shots).result()
    counts = result.get_counts()
    # Reverse the string since the most significant qubit is at the 0th index of the measurement string
    reversed_keys_dict = {}
    for key, value in counts.items():
        reversed_keys_dict[key[::-1]] = value
    return reversed_keys_dict

In [5]:
%%capture cap --no-stderr

# Encrypt 
global_params = GlobalParameters.generate_from_lambda(1)
key = Key.generate_key(global_params)
message = random_bit_string(global_params.n)
ciphertext = encrypt(message, key, global_params)
original_circuit = ciphertext.circuit.copy() # We might modify this circuit in subsequent steps

print(f"Message length: {global_params.n}")
print(f"Total number of qubits: {global_params.m}")
print(f"Qubits for deletion: {global_params.k}")
print(f"Qubits used for message encryption: {global_params.s}")

# Write states to files
with open("data/global_params.txt", "w") as f:
    f.write(global_params.to_json())

with open("data/key.txt", "w") as f:
    f.write(key.to_json())

with open("data/ciphertext.txt", "w") as f:
    f.write(ciphertext.to_json())

with open("data/message.txt", "w") as f:
    f.write(message)

with open ("data/base_circuit.qpy", "wb") as f:
    qpy_serialization.dump(original_circuit, f)

In [6]:
write_cell_to_file(cap.stdout)

In [7]:
%%capture cap --no-stderr

print("-----TEST 1: HONEST DELETION-----")
deletion_circuit_test1 = delete(ciphertext)
deletion_counts_test1 = run_and_measure(deletion_circuit_test1)
export_counts(deletion_counts_test1, csv_filename="data/test1-deletion-counts.csv", key_label="Deletion measurement")
verify_deletion_counts(deletion_counts_test1, key, global_params)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [8]:
write_cell_to_file(cap.stdout)

In [9]:
%%capture cap --no-stderr

print("-----TEST 2: DECRYPTION-----")
decryption_circuit_test2 = create_decryption_circuit(key, ciphertext)
decryption_counts_test2 = run_and_measure(decryption_circuit_test2)
export_counts(decryption_counts_test2, csv_filename="data/test2-decryption-counts.csv", key_label="Decryption measurement")
decrypt_results(decryption_counts_test2, key, ciphertext, message)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [10]:
write_cell_to_file(cap.stdout)

In [11]:
%%capture cap --no-stderr

# Test 3 - honest delete and then decrypt
print("-----TEST 3: HONEST DELETION, THEN DECRYPTION-----")
deletion_circuit_test3 = delete(ciphertext)
ciphertext.circuit = deletion_circuit_test3 # To use the new circuit as the starting point of the decryption_measurement circuit
decryption_circuit_test3 = create_decryption_circuit(key, ciphertext)
raw_counts_test3 = run_and_measure(decryption_circuit_test3)
deletion_counts_test3 = {}
decryption_counts_test3 = {}
for measurement, count in raw_counts_test3.items():
    # Qiskit will return a space-separated string of the two measurements. 
    # run_and_measure will reverse the string so that the first substring is the first measurement.
    deletion_measurement, decryption_measurement = measurement.split(" ")
    deletion_counts_test3[deletion_measurement] = deletion_counts_test3.get(deletion_measurement, 0) + count
    decryption_counts_test3[decryption_measurement] = decryption_counts_test3.get(decryption_measurement, 0) + count

export_counts(raw_counts_test3, csv_filename="data/test3-raw-counts.csv", key_label="Measurement")
export_counts(deletion_counts_test3, csv_filename="data/test3-deletion-counts.csv", key_label="Deletion measurement")
export_counts(decryption_counts_test3, csv_filename="data/test3-decryption-counts.csv", key_label="Decryption measurement")
accepted_certificates_test3 = verify_deletion_counts(deletion_counts_test3, key, global_params)
# Test the decryption results for all the measurements, regardless of whether or not the certificate of deletion was accepted
decrypt_results(decryption_counts_test3, key, ciphertext, message)

# Test the decryption results for just the measurements where the certificate of deletion was accepted
accepted_deletion_decryption_counts_test3 = {}
for measurement, count in raw_counts_test3.items():
    deletion_measurement, decryption_measurement = measurement.split(" ")
    if deletion_measurement in accepted_certificates_test3:
        accepted_deletion_decryption_counts_test3[decryption_measurement] = accepted_deletion_decryption_counts_test3.get(decryption_measurement, 0) + count

print()
print("Of the measurements where the proof of deletion was accepted, the following are the decryption statistics:")
decrypt_results(accepted_deletion_decryption_counts_test3, key, ciphertext, message)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [12]:
write_cell_to_file(cap.stdout)

In [13]:
%%capture cap --no-stderr

print("-----TEST 4: MALICIOUS DELETION, THEN DECRYPTION-----")
deletion_circuit_test4 = breidbart_measurement(ciphertext)
ciphertext.circuit = deletion_circuit_test4 # To use the new circuit as the starting point of the decryption_measurement circuit
decryption_circuit_test4 = create_decryption_circuit(key, ciphertext)
raw_counts_test4 = run_and_measure(decryption_circuit_test4)
deletion_counts_test4 = {}
decryption_counts_test4 = {}
for measurement, count in raw_counts_test4.items():
    # Qiskit will return a space-separated string of the two measurements. 
    # run_and_measure will reverse the string so that the first substring is the first measurement.
    deletion_measurement, decryption_measurement = measurement.split(" ")
    deletion_counts_test4[deletion_measurement] = deletion_counts_test4.get(deletion_measurement, 0) + count
    decryption_counts_test4[decryption_measurement] = decryption_counts_test4.get(decryption_measurement, 0) + count

export_counts(raw_counts_test4, csv_filename="data/test4-raw-counts.csv", key_label="Measurement")
export_counts(deletion_counts_test4, csv_filename="data/test4-deletion-counts.csv", key_label="Deletion measurement")
export_counts(decryption_counts_test4, csv_filename="data/test4-decryption-counts.csv", key_label="Decryption measurement")
accepted_certificates_test4 = verify_deletion_counts(deletion_counts_test4, key, global_params)
# Test the decryption results for all the measurements, regardless of whether or not the certificate of deletion was accepted
decrypt_results(decryption_counts_test4, key, ciphertext, message)

# Test the decryption results for just the measurements where the certificate of deletion was accepted
accepted_deletion_decryption_counts_test4 = {}
for measurement, count in raw_counts_test4.items():
    deletion_measurement, decryption_measurement = measurement.split(" ")
    if deletion_measurement in accepted_certificates_test4:
        accepted_deletion_decryption_counts_test4[decryption_measurement] = accepted_deletion_decryption_counts_test4.get(decryption_measurement, 0) + count

print()
print("Of the measurements where the proof of deletion was accepted, the following are the decryption statistics:")
decrypt_results(accepted_deletion_decryption_counts_test4, key, ciphertext, message)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [14]:
write_cell_to_file(cap.stdout)

In [15]:
# Create zip file
shutil.make_archive("data_export", "zip", "data");